In [1]:
import requests
import json 
from bs4 import BeautifulSoup
from lxml import html
import re
import pandas as pd 
from tqdm import tqdm

# Retrieve information from each restaurant 

In [2]:
def extract_restaurant(url, header):
    response = requests.get(url, headers=header)
    
    # parsing 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    tree = html.fromstring(response.content)
    
    #content 
    restaurant_page = {}
    restaurant_page['url'] = url

    # restaurant element
    # title 
    rest_title = ''
    try :
    # rest_title = html_soup.find_all('h1')[-1].text
        rest_title = tree.xpath('/html/body/div[1]/div/main/div/section[3]/section/section/div/div/div/h1')[0].text
        restaurant_page['title'] = rest_title
    except :
        pass 
    
    # extract restaurant type 
    rest_type = []
    for item in html_soup.find_all('span'): 
        try : 
            rest_type.append((item.find_all('a', href=True)[0].text))
        except : 
            pass 
    rest_type = '|'.join(rest_type[:-1])
    restaurant_page['rest_type'] = rest_type

    # price 
    rest_price = []
    
    for  item in html_soup.find_all('p'):
        if 'Rp' in item.text:
            rest_price.append(item.text)
    
    rest_price = '|'.join(rest_price)
    restaurant_page['rest_price'] = rest_price

    # facility 
    rest_facility = []
    
    for item in html_soup.find_all('p', {'class': re.compile(r'cunMUz')}):
        rest_facility.append(item.text)
    restaurant_page['rest_facility'] = '|'.join(rest_facility)

    # long lat 
    long_lat = [] 
    try : 
        for item in html_soup.find_all('a',href=True):
            if 'google' in item['href'] : 
                long_lat.append(item['href'])
            else : 
                pass 
    except :
        pass
        
    restaurant_page['rest_longlat'] = list(set(long_lat))

    
    # rating 
    rating = ''
    try : 
        rating = html_soup.find_all("div", {"class": "sc-1q7bklc-1 cILgox"})[0].text
        restaurant_page['rating'] = rating
    except : 
        pass 
    
    # total review 
    review = ''
    # review = tree.xpath('//div[normalize-space(text())="Reviews"]/preceding-sibling::div')

    try :
        review = tree.xpath('//*[@id="root"]/div/main/div/section[3]/section/section/div/div/div/section/div/div[2]/div[1]')
        review = review[0].text
        restaurant_page['review'] = review
    except : 
        restaurant_page['review'] = review
        
    return restaurant_page

In [3]:
# url = 'https://www.zomato.com/jakarta/monks-kelapa-gading'
url = 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info'
my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0", 
          "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
              "Accept-Language":"en-US,en;q=0.5", 
             "Accept-Encoding":"gzip, deflate, br"}

rest_content = extract_restaurant(url, my_headers)

In [4]:
rest_content

{'url': 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info',
 'title': 'Aroma Palopo',
 'rest_type': 'Makassar',
 'rest_price': 'Rp80.000 for two people (approx.)',
 'rest_facility': 'Home Delivery|Takeaway Available|Indoor Seating|Smoking Area|No Alcohol Available',
 'rest_longlat': ['https://www.google.com/maps/dir/?api=1&destination=-6.1714440000,106.8982310000'],
 'rating': '3.7',
 'review': '20'}

In [30]:
df = pd.read_csv('1_jakarta_rest.csv')
df = df.dropna()
df = df.drop_duplicates()
df2 = pd.read_csv('3_jak_rest_detail.csv')

In [32]:
df[df['rest_url']=='https://www.zomato.com/jakarta/namelaka-menteng/info']

,district,rest_url,count
1473,https://www.zomato.com/jakarta/thamrin-restaur...,https://www.zomato.com/jakarta/namelaka-menten...,501
6509,https://www.zomato.com/jakarta/setiabudi-resta...,https://www.zomato.com/jakarta/namelaka-menten...,501
8592,https://www.zomato.com/jakarta/menteng-restaur...,https://www.zomato.com/jakarta/namelaka-menten...,501
9700,https://www.zomato.com/jakarta/cikini-restaurants,https://www.zomato.com/jakarta/namelaka-menten...,489
9981,https://www.zomato.com/jakarta/menteng-restaur...,https://www.zomato.com/jakarta/namelaka-menten...,9
10127,https://www.zomato.com/jakarta/cikini-restaurants,https://www.zomato.com/jakarta/namelaka-menten...,501


In [31]:
df.shape, df2.shape

((10558, 3), (7305, 8))

In [22]:
df[~df.rest_url.isin(df2['url'])]

,district,rest_url,count


In [23]:
df['rest_url'].value_counts()

https://www.zomato.com/jakarta/namelaka-menteng/info                            6
https://www.zomato.com/jakarta/bunga-rampai-menteng/info                        6
https://www.zomato.com/jakarta/al-jazeerah-signature-menteng/info               6
https://www.zomato.com/jakarta/maple-oak-menteng/info                           6
https://www.zomato.com/jakarta/tugu-kunstkring-paleis-menteng/info              6
                                                                               ..
https://www.zomato.com/jakarta/warung-sate-solo-pondok-sri-pondok-indah/info    1
https://www.zomato.com/jakarta/onezo-tanjung-duren/info                         1
https://www.zomato.com/jakarta/komuja-coffee-pantai-indah-kapuk/info            1
https://www.zomato.com/jakarta/koi-teppanyaki-pondok-indah/info                 1
https://www.zomato.com/jakarta/imperial-kitchen-dimsum-tebet/info               1
Name: rest_url, Length: 7277, dtype: int64

In [29]:
df['rest_url'].value_counts()

https://www.zomato.com/jakarta/namelaka-menteng/info                            6
https://www.zomato.com/jakarta/bunga-rampai-menteng/info                        6
https://www.zomato.com/jakarta/al-jazeerah-signature-menteng/info               6
https://www.zomato.com/jakarta/maple-oak-menteng/info                           6
https://www.zomato.com/jakarta/tugu-kunstkring-paleis-menteng/info              6
                                                                               ..
https://www.zomato.com/jakarta/warung-sate-solo-pondok-sri-pondok-indah/info    1
https://www.zomato.com/jakarta/onezo-tanjung-duren/info                         1
https://www.zomato.com/jakarta/komuja-coffee-pantai-indah-kapuk/info            1
https://www.zomato.com/jakarta/koi-teppanyaki-pondok-indah/info                 1
https://www.zomato.com/jakarta/imperial-kitchen-dimsum-tebet/info               1
Name: rest_url, Length: 7277, dtype: int64

In [11]:
df3 = pd.merge(df,df2['url'],left_on='rest_url',right_on='url',how='left')

In [16]:
df3.head()

,district,rest_url,count,url
0,https://www.zomato.com/jakarta/kelapa-gading-r...,https://www.zomato.com/jakarta/wakacao-1-kelap...,501,https://www.zomato.com/jakarta/wakacao-1-kelap...
1,https://www.zomato.com/jakarta/kelapa-gading-r...,https://www.zomato.com/jakarta/the-coffee-bean...,501,https://www.zomato.com/jakarta/the-coffee-bean...
2,https://www.zomato.com/jakarta/kelapa-gading-r...,https://www.zomato.com/jakarta/angke-kelapa-ga...,501,https://www.zomato.com/jakarta/angke-kelapa-ga...
3,https://www.zomato.com/jakarta/kelapa-gading-r...,https://www.zomato.com/jakarta/ikan-nila-pak-u...,501,https://www.zomato.com/jakarta/ikan-nila-pak-u...
4,https://www.zomato.com/jakarta/kelapa-gading-r...,https://www.zomato.com/jakarta/hong-kong-sheng...,501,https://www.zomato.com/jakarta/hong-kong-sheng...


In [17]:
df3.shape

(10646, 4)

In [18]:
df3.isnull().sum()

district    0
rest_url    0
count       0
url         0
dtype: int64

In [46]:
df.iloc[5223:,:]

,district,rest_url,count
5224,https://www.zomato.com/jakarta/fatmawati-resta...,https://www.zomato.com/jakarta/sgd-tofu-house-...,501
5225,https://www.zomato.com/jakarta/fatmawati-resta...,https://www.zomato.com/jakarta/house-of-omuric...,501
5226,https://www.zomato.com/jakarta/fatmawati-resta...,https://www.zomato.com/jakarta/waroeng-modus-1...,501
5227,https://www.zomato.com/jakarta/fatmawati-resta...,https://www.zomato.com/jakarta/white-tree-caf%...,501
5228,https://www.zomato.com/jakarta/fatmawati-resta...,https://www.zomato.com/jakarta/pink-pink-fatma...,501
...,...,...,...
10555,https://www.zomato.com/jakarta/dharmawangsa-re...,https://www.zomato.com/jakarta/porter-bar-kema...,501
10556,https://www.zomato.com/jakarta/dharmawangsa-re...,https://www.zomato.com/jakarta/amertha-warung-...,501
10557,https://www.zomato.com/jakarta/dharmawangsa-re...,https://www.zomato.com/jakarta/izakaya-taichan...,501
10558,https://www.zomato.com/jakarta/dharmawangsa-re...,https://www.zomato.com/jakarta/soto-ayam-amben...,501


In [47]:
result = []
for url in tqdm(df.iloc[5223:,:].itertuples()):
    result.append(extract_restaurant(url.rest_url, my_headers))

5335it [3:32:04,  2.39s/it]


In [48]:
df_result = pd.DataFrame(result)

In [49]:
df_result.tail()

,url,title,rest_type,rest_price,rest_facility,rest_longlat,rating,review
5330,https://www.zomato.com/jakarta/porter-bar-kema...,Porter Bar,Western|Finger Food,Rp200.000 for two people (approx.) Without alc...,Takeaway Available|Full Bar Available|Smoking ...,[https://www.google.com/maps/dir/?api=1&destin...,3.6,93
5331,https://www.zomato.com/jakarta/amertha-warung-...,Amertha Warung Coffee,Coffee|Indonesian,Rp160.000 for two people (approx.),Breakfast|Takeaway Available|Valet Parking Ava...,[https://www.google.com/maps/dir/?api=1&destin...,3.8,45
5332,https://www.zomato.com/jakarta/izakaya-taichan...,Izakaya Taichan,Japanese|Ramen|Sushi,Rp300.000 for two people (approx.) Without alc...,Takeaway Available|Wine and Beer|Indoor Seatin...,[https://www.google.com/maps/dir/?api=1&destin...,3.5,65
5333,https://www.zomato.com/jakarta/soto-ayam-amben...,Soto Ayam Ambengan Pak Sadi (ASLI),Soto|Jawa,Rp150.000 for two people (approx.),Breakfast|Takeaway Available|Wifi|Indoor Seati...,[https://www.google.com/maps/dir/?api=1&destin...,4.0,96
5334,https://www.zomato.com/jakarta/sapo-oriental-b...,Sapo Oriental,Chinese,Rp200.000 for two people (approx.),Home Delivery|Takeaway Available|Indoor Seatin...,[https://www.google.com/maps/dir/?api=1&destin...,4.2,137


In [50]:
df_result.to_csv('jakarta_restaurant_detail3.csv',index=False)

In [ ]:
%%time
df['rest_info'] = df['rest_url'].apply(extract_restaurant, header = my_headers)

In [52]:
# set url 
url = 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info'
response = requests.get(url, headers=my_headers)

# parsing 
html_soup = BeautifulSoup(response.text, 'html.parser')
tree = html.fromstring(response.content)

In [54]:
# testing area for xpath 
rating = tree.xpath('//div[normalize-space(text())="Reviews"]/preceding-sibling::div')

# tes = tree.xpath('/html/body/div[1]/div/main/div/section[4]/section/article/section/div[2]/a')

price = tree.xpath('//*[@id="root"]/div[2]/main/div/section[4]/section/section/article[1]/section[2]/p[2]')
# help(tes[0])
# print(tes[0].get_attribute("value"))

facility = tree.xpath('//*[@id="root"]/div[2]/main/div/section[4]/section/section/article[1]/section[2]/h3[4]')

In [55]:
# rating[0].text
for item in html_soup.find_all('p', {'class': re.compile(r'cunMUz')}):
    print (item.text)
# sc-1hez2tp-0 cunMUz

Home Delivery
Takeaway Available
Indoor Seating
Smoking Area
No Alcohol Available


In [16]:
main_url = 'https://www.zomato.com/jakarta'
response = requests.get(main_url, headers=my_headers)

# parsing 
html_soup = BeautifulSoup(response.text, 'html.parser')
tree = html.fromstring(response.content)

In [17]:
# extract 

rest_jakarta = tree.xpath('/html/body/div[1]/div/div[4]/div[4]/div[2]/div')

In [42]:
for item in rest_jakarta[0][0]:
#     print(item.tag)
    for subitem in item:
        print(subitem.tag)

div


In [43]:
print(rest_jakarta)

[<Element div at 0x1e07c79e0e8>]
